In [1]:
import io
import re
import csv
import time
import xlwt
import yaml
import math
import random
import numpy as np
import pandas as pd
from xlwt import Workbook
from datetime import datetime, date

In [2]:
def readAiContentData():
    cityData = pd.read_csv("cityData.csv")
    return cityData

In [3]:
# Dump Data to yaml 
content = pd.read_csv("OpeningParagraphTest.csv")
header = ["related_metric", "data_ranking","Python Copy"]
orderList = [""]*len(header)
dumpData = {}
    
for j in range(len(content)):
    ai_content = content.loc[j,header].values.flatten().tolist()
    for i in range(len(ai_content)):
        try: 
            if math.isnan(ai_content[i]):
                ai_content[i] = ""
        except:
            orderList[i] = ai_content[i]
        if ai_content[i] != "" and i != len(orderList) - 1:
            if i == 0:
                dumpData[orderList[i]] = {}
                key = orderList[i]
                continue
            dumpData[key].update({orderList[i]: ai_content[i + 1]})


with open(r'content.yml', 'w') as file:
    documents = yaml.dump(dumpData, file)



In [4]:
# Create the dictionary and create ranking columns
def meansort(attribute ,attributeList):
    columnMean = round(attributeList[attribute].mean(),2)
    x = attributeList.reset_index()[attributeList.columns].values.tolist()
    return sorted(x, key=lambda x: abs(x[1] - columnMean), reverse = True)


def createRankColumns(cityData):
    count = 0
    neutralListMap = {}
    mydict = {}

    
    with open('RankingConfigTest.csv', mode='r') as infile:
        reader = csv.reader(infile)
        next(reader)
        mydict = {rows[0].strip():[rows[1], rows[2], rows[3]] for rows in reader}
        infile.close()
    print(len(mydict))
    print(mydict)

    matrixList = cityData.iloc[:, 5:55]
    for attribute in matrixList.columns:
        count+=1
        if (attribute in mydict.keys()) and (mydict[attribute.strip()][0] != "N/A"):
            cityData[attribute+"_rank"] = cityData[attribute].rank(ascending = mydict[attribute][1] == "FALSE", method="dense")
            if mydict[attribute][1] == "N/A":
                cityData[attribute].fillna(value=cityData[attribute].mean(), inplace=True)
#                 cityData[attribute+"_rank"] = cityData[attribute].rank(ascending = mydict[attribute][1] == "FALSE")
                neutralListMap[attribute] = meansort(attribute, cityData[["city", attribute]])
    return mydict, neutralListMap
# print(neutralListMap["avg. rainfall per year (mm)"])



In [5]:
# cityData[["city","avg. rainfall per year (mm)_rank"]]

In [6]:
# Given the attribute, buckets corresponding to the attribue and the city ranking corresponding to the attribute identify the bucket the data falls in

def fetchkey(cityData, att, bucketList, rank, mydict):
    bucketList = list(bucketList)
    res = "".join([i for i in bucketList if "middle" in i])
    op = ""
    flag = True
    for i in range(len(bucketList)):
        print("Identify bucket for attribut = ", att, "Attribute Buckets = ", bucketList, "CityRank for the attribute = ",rank)
        if att == "air quality":
            return fetchAirQualityKey(rank)
        if not bucketList[i].endswith("%"):
            if int(rank) < 10:
                return bucketList[i]
#             else:
#                 return bucketList[i]
        else:
            if (res != "" and flag and mydict[att][1] != "N/A"):
                flag = False
                op = res
            if bucketList[i].startswith("top"):
                percentCheck = bucketList[i].replace("top", "").replace("%", "")
                columnRankCount = cityData[att].nunique()
                endRank = (int(percentCheck)/100)*columnRankCount
                print("Rank = ",rank, "Unique rank count = ", columnRankCount,"range", endRank,"precent", percentCheck)
                if rank <= endRank:
                    op = bucketList[i]
            elif(bucketList[i].startswith("bottom")):
                percentCheck = bucketList[i].replace("bottom", "").replace("%", "")
                columnRankCount = cityData[att].nunique()
                endRank = (int(percentCheck)/100)*columnRankCount
                print("rank", rank, "endRank = ", endRank, "endRank = ", endRank, "columnRankCount", columnRankCount, "difference check", columnRankCount - endRank)
                if (rank >= (columnRankCount - endRank)):
                    op = bucketList[i]
    return op
                
def fetchAirQualityKey(rank):
    if rank >= 1 and rank <=50:
        return "0-50"
    if rank >= 51 and rank <=100:
        return "51-100"
    if rank >= 101 and rank <=150:
        return "101-150"
    if rank >= 151 and rank <=200:
        return "151-200"
    if rank >= 201 and rank <=300:
        return "201-300"
    if rank >= 301 and rank <= 500:
        return "301-500"
    

In [7]:

def replace(city, stateId, att, sentence, rank):
    print("From replace")
    sentence = sentence.replace("{rank}", numToOrdinal(rank))
    sentence = sentence.replace("{inverse_rank}", numToOrdinal(cityData[att+"_rank"].nunique() - rank + 1))
    print("From replace")
    keywordList = re.findall("[{][\w.,$&-/(/)/\s]{0,}[}]", sentence)
    print(keywordList)
    for keyword in keywordList:
        print("keyword is this", keyword)
        dataToReplace = cityData[np.logical_and(cityData["city"] == city, cityData["state_id"] == stateId)][keyword[1:-1]].values[0]
        print("*****************", dataToReplace)
        if not isinstance(dataToReplace, str):
            dataToReplace = numberFormat(dataToReplace)
        if "$" in keyword:
            dataToReplace = "$" + dataToReplace
        sentence = sentence.replace(keyword, dataToReplace)
    return sentence

def numberFormat(num):
    if not isinstance(num, float):
        num = str("{:,}".format(int(num)))
    else:
        num = ("{:,}".format(num)).rstrip("0").rstrip(".")
    return num

def numToOrdinal(value):
    if value % 100//10 != 1:
        if value % 10 == 1:
            ordval = u"%d%s" % (value, "st")
        elif value % 10 == 2:
            ordval = u"%d%s" % (value, "nd")
        elif value % 10 == 3:
            ordval = u"%d%s" % (value, "rd")
        else:
            ordval = u"%d%s" % (value, "th")
    else:
        ordval = u"%d%s" % (value, "th")

    return ordval



In [8]:
def fetchSentences(city, stateId, att, rank, key, descriptionData):
    print("key to check from the yaml for attribute = ", att," and key = ", key)
    sentenceList = descriptionData[att][key].split("|")
    sentence = random.choice(sentenceList)
    sentence = replace(city, stateId, att ,sentence, rank)  
    return sentence

In [9]:
def generateParagraph(city, stateId, metricRankList, cityParagraph, mydict):
    cityParagraph = ""
    bucketKeyList = []
    with open("content.yml") as file:
        descriptionData = yaml.full_load(file)
    for i in range(len(metricRankList)):
        if math.isnan(metricRankList[i][1]):
            continue
        key = fetchkey(cityData, metricRankList[i][0], descriptionData[metricRankList[i][0]].keys(), metricRankList[i][1], mydict)
        if key != "":
            res = fetchSentences(city, stateId, metricRankList[i][0], metricRankList[i][1], key, descriptionData)
            print("\n", "Random sentence that was choosen ", res, "\n")
            bucketKeyList.append(key)
#             cityParagraph += res.strip() + " "
            cityParagraph += res.replace("\n", "") + " " 
            res =""
#                 res =""
    return cityParagraph, bucketKeyList

In [10]:
start_time = time.time()
row = 1
wb = Workbook()
sheet1 = wb.add_sheet('Sheet 1')
cityData = readAiContentData()
mydict, neutralListMap = createRankColumns(cityData)
for cityIndex in range(len(cityData)):
    citylistData = (cityData.iloc[[cityIndex]]).values
    city = citylistData[0][1]
    stateId = citylistData[0][2]
    col = 0
#     if city != "Fairbanks":
#         continue
    posAttributeList = []
    negAttributeList = []
    neutralAttributeList = []
    fixedList = []
    counter = 0
    citycontent = cityData[np.logical_and(cityData["city"] == city, cityData["state_id"] == stateId)]
    rankColumns = cityData.iloc[:, 61:]
    rankData = citycontent[rankColumns.columns].values
    for i in rankColumns.columns:
#         print("^^^^^^^^^^^^^^^",i)
        metric = i.replace("_rank", "")
        metrixData = dict(zip(cityData["city"], cityData[metric]))
        if metric not in mydict:
            counter += 1
            continue
        if pd.isna(metrixData[city]):
            counter += 1
            continue
#         if metric == "2019 job opportunity":
        if metric == "2019 job opportunity_normalized":
            fixedList.append((metric, rankData[0][counter]))
        elif mydict[metric][1] == "TRUE":
            posAttributeList.append((metric, rankData[0][counter]))
        elif mydict[metric][1] == "FALSE":
            negAttributeList.append((metric, rankData[0][counter]))
        elif mydict[metric][1] == "N/A":
            neutralAttributeList.append((metric, neutralListMap[metric].index([city, metrixData[city]]) + 1))
        counter += 1
    print("\n ----------------------------------- Lists ----------------------------------------------- \n")
    print("City in consideration", city, citylistData[0][2])
    print("Fixed List", fixedList)
    print("Pos List", posAttributeList)
    print("Neg List", negAttributeList)
    print("Neu List", neutralAttributeList)
    print("\n ----------------------------------- Lists ----------------------------------------------- \n")
    fixedList.extend(sorted(posAttributeList, key=lambda x: x[1])[0:2])
    fixedList.extend(sorted(negAttributeList, key=lambda x: x[1])[0:1])
    fixedList.extend(sorted(neutralAttributeList, key=lambda x: x[1])[0:1])
    print("\n Final list of attributes \n", fixedList, "\n\n")
    fixedList[-1] = (fixedList[-1][0] , cityData[np.logical_and(cityData["city"] == city, cityData["state_id"] == stateId)][fixedList[-1][0]+"_rank"].values[0])
    print("\n Final list of attributes \n", fixedList, "\n\n")
    cityDescriptionParagraph, bucketKeyList = generateParagraph(city, citylistData[0][2] ,fixedList, "", mydict)
    print("################################################################## \n\n" ,row,city, cityDescriptionParagraph, "\n\n ##################################################################")
    sheet1.write(row, col, city)
    col += 1
    sheet1.write(row, col, cityDescriptionParagraph)
    col += 1
    sheet1.write(row, col, stateId)
    for i in range(len(bucketKeyList)):
        col += 1
        sheet1.write(row, col, bucketKeyList[i])
    row += 1
wb.save('generatedParagraphs.xls')
endTime = datetime.now()

print("--- %s Script Runtime  ---" % (time.time() - start_time))




25
{'air quality': ['yes', 'FALSE', 'B'], 'avg. rainfall per year (inch)': ['yes', 'N/A', 'C'], 'avg. snow per year (inch)': ['yes', 'N/A', 'C'], 'commute time (min)': ['yes', 'FALSE', 'A'], 'density': ['yes', 'N/A', 'B'], 'expend, per student $': ['top', 'TRUE', 'C'], 'GDP': ['yes', 'TRUE', 'B'], 'Good&service index': ['yes', 'TRUE', 'A'], 'Grocery Index': ['yes', 'TRUE', 'A'], 'Healthcare index': ['yes', 'TRUE', 'A'], 'Housing Index': ['yes', 'TRUE', 'A'], '2019 job opportunity_normalized': ['yes', 'TRUE', 'A'], 'median age': ['bottom', 'N/A', 'C'], 'median home value': ['yes', 'FALSE', 'B'], 'median home value growth percentage': ['N/A', 'N/A', 'B'], 'median household income': ['N/A', 'TRUE', 'B'], 'population': ['yes', 'N/A', 'A'], 'population growth percentage': ['N/A', 'N/A', 'B'], 'poverty rate': ['yes', 'FALSE', 'B'], 'school_record_normalized': ['top', 'TRUE', 'C'], 'six figure household percentage': ['yes', 'TRUE', 'C'], 'summer high': ['yes', 'N/A', 'C'], 'unemployment rate'

Identify bucket for attribut =  2019 job opportunity_normalized Attribute Buckets =  ['bottom25%', 'middle50%', 'top25%'] CityRank for the attribute =  100.0
rank 100.0 endRank =  45.5 endRank =  45.5 columnRankCount 182 difference check 136.5
Identify bucket for attribut =  2019 job opportunity_normalized Attribute Buckets =  ['bottom25%', 'middle50%', 'top25%'] CityRank for the attribute =  100.0
Identify bucket for attribut =  2019 job opportunity_normalized Attribute Buckets =  ['bottom25%', 'middle50%', 'top25%'] CityRank for the attribute =  100.0
Rank =  100.0 Unique rank count =  182 range 45.5 precent 25
key to check from the yaml for attribute =  2019 job opportunity_normalized  and key =  middle50%
From replace
From replace
['{2019 job opportunity}', '{city}']
keyword is this {2019 job opportunity}
***************** 59000.0
keyword is this {city}
***************** Albany

 Random sentence that was choosen  
The number of job openings here is okay at 59,000, putting Albany 10

From replace
['{unemployment rate}', '{city}']
keyword is this {unemployment rate}
***************** 5.6
keyword is this {city}
***************** Albuquerque

 Random sentence that was choosen  
The 5.6% unemployment rate in Albuquerque is the 29th lowest and the 50th highest on our city list. 

Identify bucket for attribut =  avg. rainfall per year (inch) Attribute Buckets =  ['bottom25%', 'middle50%', 'top15%'] CityRank for the attribute =  158.0
rank 158.0 endRank =  43.75 endRank =  43.75 columnRankCount 175 difference check 131.25
Identify bucket for attribut =  avg. rainfall per year (inch) Attribute Buckets =  ['bottom25%', 'middle50%', 'top15%'] CityRank for the attribute =  158.0
Identify bucket for attribut =  avg. rainfall per year (inch) Attribute Buckets =  ['bottom25%', 'middle50%', 'top15%'] CityRank for the attribute =  158.0
Rank =  158.0 Unique rank count =  175 range 26.25 precent 15
key to check from the yaml for attribute =  avg. rainfall per year (inch)  and key =

Identify bucket for attribut =  expend, per student $ Attribute Buckets =  ['bottom25%', 'middle50%', 'top25%'] CityRank for the attribute =  31.0
rank 31.0 endRank =  61.0 endRank =  61.0 columnRankCount 244 difference check 183.0
Identify bucket for attribut =  expend, per student $ Attribute Buckets =  ['bottom25%', 'middle50%', 'top25%'] CityRank for the attribute =  31.0
Identify bucket for attribut =  expend, per student $ Attribute Buckets =  ['bottom25%', 'middle50%', 'top25%'] CityRank for the attribute =  31.0
Rank =  31.0 Unique rank count =  244 range 61.0 precent 25
key to check from the yaml for attribute =  expend, per student $  and key =  top25%
From replace
From replace
['{expend, per student $}', '{city}']
keyword is this {expend, per student $}
***************** 15847.8
keyword is this {city}
***************** Allentown

 Random sentence that was choosen  The average amount spent per student is $15,847.8, ranking Allentown a high 31st on our city list in terms of ed

Identify bucket for attribut =  2019 job opportunity_normalized Attribute Buckets =  ['bottom25%', 'middle50%', 'top25%'] CityRank for the attribute =  101.0
Rank =  101.0 Unique rank count =  182 range 45.5 precent 25
key to check from the yaml for attribute =  2019 job opportunity_normalized  and key =  middle50%
From replace
From replace
['{2019 job opportunity}', '{city}']
keyword is this {2019 job opportunity}
***************** 44900.0
keyword is this {city}
***************** Ames

 Random sentence that was choosen  
There are currently about 44,900 employment opportunities in Ames.  

Identify bucket for attribut =  GDP Attribute Buckets =  ['top25%'] CityRank for the attribute =  48.0
Rank =  48.0 Unique rank count =  193 range 48.25 precent 25
key to check from the yaml for attribute =  GDP  and key =  top25%
From replace
From replace
['{city}', '{GDP}']
keyword is this {city}
***************** Ames
keyword is this {GDP}
***************** -0.77

 Random sentence that was choose

...

rank 3.0 endRank =  98.25 endRank =  98.25 columnRankCount 131 difference check 32.75
Identify bucket for attribut =  median age Attribute Buckets =  ['bottom75%', 'top25%'] CityRank for the attribute =  3.0
Rank =  3.0 Unique rank count =  131 range 32.75 precent 25
key to check from the yaml for attribute =  median age  and key =  top25%
From replace
From replace
['{city}', '{median age}']
keyword is this {city}
***************** Vero Beach
keyword is this {median age}
***************** 53.4

 Random sentence that was choosen  Vero Beach's population's median age is 53.4, higher than 129th of the other cities on our list.  

################################################################## 

 246 Vero Beach Vero Beach has a large volume of job opportunities, with about 52,950 positions available.  Vero Beach is a wealthy city, whose GDP grew by 0.33% year over year.  Housing in Vero Beach is reasonably priced. The index is 82.7, putting it 73rd on our list of cities.  Vero Beach's p

Identify bucket for attribut =  2019 job opportunity_normalized Attribute Buckets =  ['bottom25%', 'middle50%', 'top25%'] CityRank for the attribute =  48.0
rank 48.0 endRank =  45.5 endRank =  45.5 columnRankCount 182 difference check 136.5
Identify bucket for attribut =  2019 job opportunity_normalized Attribute Buckets =  ['bottom25%', 'middle50%', 'top25%'] CityRank for the attribute =  48.0
Identify bucket for attribut =  2019 job opportunity_normalized Attribute Buckets =  ['bottom25%', 'middle50%', 'top25%'] CityRank for the attribute =  48.0
Rank =  48.0 Unique rank count =  182 range 45.5 precent 25
key to check from the yaml for attribute =  2019 job opportunity_normalized  and key =  middle50%
From replace
From replace
['{2019 job opportunity}', '{city}']
keyword is this {2019 job opportunity}
***************** 88610.0
keyword is this {city}
***************** Waterloo

 Random sentence that was choosen  
The number of job openings here is okay at 88,610, putting Waterloo 48t

Identify bucket for attribut =  school_record_normalized Attribute Buckets =  ['bottom25%', 'middle50%', 'top25%'] CityRank for the attribute =  39.0
rank 39.0 endRank =  48.75 endRank =  48.75 columnRankCount 195 difference check 146.25
Identify bucket for attribut =  school_record_normalized Attribute Buckets =  ['bottom25%', 'middle50%', 'top25%'] CityRank for the attribute =  39.0
Identify bucket for attribut =  school_record_normalized Attribute Buckets =  ['bottom25%', 'middle50%', 'top25%'] CityRank for the attribute =  39.0
Rank =  39.0 Unique rank count =  195 range 48.75 precent 25
key to check from the yaml for attribute =  school_record_normalized  and key =  top25%
From replace
From replace
['{city}', '{school_record}']
keyword is this {city}
***************** Westminster
keyword is this {school_record}
***************** 36.0

 Random sentence that was choosen  
Westminster is 39th on our list of cities in terms of education access. With 36 schools, there are plenty of edu

Identify bucket for attribut =  school_record_normalized Attribute Buckets =  ['bottom25%', 'middle50%', 'top25%'] CityRank for the attribute =  4.0
rank 4.0 endRank =  48.75 endRank =  48.75 columnRankCount 195 difference check 146.25
Identify bucket for attribut =  school_record_normalized Attribute Buckets =  ['bottom25%', 'middle50%', 'top25%'] CityRank for the attribute =  4.0
Identify bucket for attribut =  school_record_normalized Attribute Buckets =  ['bottom25%', 'middle50%', 'top25%'] CityRank for the attribute =  4.0
Rank =  4.0 Unique rank count =  195 range 48.75 precent 25
key to check from the yaml for attribute =  school_record_normalized  and key =  top25%
From replace
From replace
['{city}', '{school_record}']
keyword is this {city}
***************** Wilmington
keyword is this {school_record}
***************** 54.0

 Random sentence that was choosen  
Wilmington is 4th on our list of cities in terms of education access. With 54 schools, there are plenty of education o

Identify bucket for attribut =  2019 job opportunity_normalized Attribute Buckets =  ['bottom25%', 'middle50%', 'top25%'] CityRank for the attribute =  76.0
rank 76.0 endRank =  45.5 endRank =  45.5 columnRankCount 182 difference check 136.5
Identify bucket for attribut =  2019 job opportunity_normalized Attribute Buckets =  ['bottom25%', 'middle50%', 'top25%'] CityRank for the attribute =  76.0
Identify bucket for attribut =  2019 job opportunity_normalized Attribute Buckets =  ['bottom25%', 'middle50%', 'top25%'] CityRank for the attribute =  76.0
Rank =  76.0 Unique rank count =  182 range 45.5 precent 25
key to check from the yaml for attribute =  2019 job opportunity_normalized  and key =  middle50%
From replace
From replace
['{2019 job opportunity}', '{city}']
keyword is this {2019 job opportunity}
***************** 93570.0
keyword is this {city}
***************** Yakima

 Random sentence that was choosen  
There are currently about 93,570 employment opportunities in Yakima.  

I

In [11]:
# print(rankColumns.columns)
# print(rankData)

In [12]:
# count = 0
# start = 0
# end = len(rankColumns) - 1
# bucketList = []
# cityParagraph = ""
# res = ""
# with open("content.yml") as file:
#     descriptionData = yaml.full_load(file)

# while(start < len(rankColumns) and start < end):
# #     if count == 4:
# #         print("ran break successfully")
# #         break
# #     if count < 3:
# # #         print(rankColumns)
# #         positiveAttribute = rankColumns[start][1].replace("_rank", "")
#         if (mydict[positiveAttribute][0] != "N/A") and (mydict[positiveAttribute][0] == "top" or mydict[positiveAttribute][0] == "yes"):  
#             print(type(descriptionData[positiveAttribute].keys()))
#             key = fetchkey(rankColumns[start], descriptionData[positiveAttribute].keys(), rankColumns[start][0])
#             print("returned key from the fetch function is =======", key)
#             if key != None:
#                 count += 1
#                 res = fetchSentences(positiveAttribute, key, descriptionData)
#         start += 1
        
#     else:
#         negativeAttribute = rankColumns[end][1].replace("_rank", "")
#         if mydict[positiveAttribute][0] != "N/A" and mydict[negativeAttribute][0] != "top":
#             print("*****",rankColumns[end])
#             print("****",descriptionData[negativeAttribute].keys())
#             print("***",rankData[end])
#             key = fetchkey(rankColumns[end], descriptionData[negativeAttribute].keys(), rankColumns[start][0])
#             if key != None:
#                 count += 1
#                 res = fetchSentences(negativeAttribute, key, descriptionData)
#         end -= 1
#     cityParagraph += res
#     res =""        


# print("###############################################################")
# print(cityParagraph)
# print("###############################################################")

In [13]:
# cityData["avg. snow per year_rank"].nunique()

In [14]:
# New York
# City_name's average commute time is pretty long: about 23.1 minutes. This is the 23rd longest commute time on our city list.City_name is a small city. It has only xpopulation people living in it, ranking it as the xrank smallest population in our city list. There are 552 schools per 100,000 people, making City_name a place with good access to education. This ranks the city 58th on our city list. City_name's winter is very cold. The average temperature in winter is just XX°F, ranking this as the 42nd coldest winter city. 

In [15]:
# ('commute time_rank', 'population_rank', 'schools per 100,000 people_rank', 'median home value_rank', 'expend, per student_rank', 'avg. rainfall per year_rank', 'air quality_rank', 'Job opportunity_rank', 'unemployment rate_rank', 'median age_rank', 'poverty rate_rank', 'Healthcare index_rank', 'Utilities index_rank', 'Good&service index_rank', 'GDP_rank', 'avg. snow per year_rank', 'Housing Index_rank', 'median household income growth_rank')
# (1.0, 1.0, 1.0, 3.0, 7.0, 12.0, 14.0, 43.0, 43.0, 47.0, 82.0, 108.0, 113.0, 115.0, 124.0, 124.0, 145.0, 159.0)


# ('Utilities index_rank', 'GDP_rank', 'Good&service index_rank', 'Healthcare index_rank', 'avg. snow per year_rank', 'schools per 100,000 people_rank', 'Job opportunity_rank', 'unemployment rate_rank', 'Housing Index_rank', 'median household income growth_rank', 'poverty rate_rank', 'median age_rank', 'median home value_rank', 'commute time_rank', 'avg. rainfall per year_rank', 'population_rank', 'expend, per student_rank', 'air quality_rank')
# (3.0, 13.0, 24.0, 27.0, 37.0, 64.0, 69.0, 69.0, 83.0, 92.0, 93.0, 94.0, 109.0, 115.0, 144.0, 152.0, 196.0, nan)

# ('air quality_rank', 'Job opportunity_rank', 'unemployment rate_rank', 'avg. snow per year_rank', 'avg. rainfall per year_rank', 'schools per 100,000 people_rank', 'poverty rate_rank', 'median age_rank', 'expend, per student_rank', 'commute time_rank', 'population_rank', 'Healthcare index_rank', 'Good&service index_rank', 'median household income growth_rank', 'Housing Index_rank', 'Utilities index_rank', 'GDP_rank', 'median home value_rank')
# (19.0, 26.0, 26.0, 27.0, 38.0, 43.0, 44.0, 45.0, 55.0, 60.0, 65.0, 66.0, 75.0, 75.0, 82.0, 83.0, 121.0, 135.0)

# ('Job opportunity_rank', 'unemployment rate_rank', 'poverty rate_rank', 'air quality_rank', 'expend, per student_rank', 'Housing Index_rank', 'median age_rank', 'Good&service index_rank', 'Utilities index_rank', 'schools per 100,000 people_rank', 'avg. rainfall per year_rank', 'median household income growth_rank', 'Healthcare index_rank', 'commute time_rank', 'avg. snow per year_rank', 'median home value_rank', 'GDP_rank', 'population_rank')
# (4.0, 4.0, 14.0, 24.0, 43.0, 51.0, 70.0, 72.0, 76.0, 79.0, 85.0, 89.0, 92.0, 95.0, 122.0, 129.0, 161.0, 174.0)


In [16]:
# Test for preprocessing

# cityData.head()
# cityData["avg. rainfall per year"]
# cityData["avg. snow per year"]
# empDfObj = pd.DataFrame(cityData, columns=["avg. rainfall per year", "avg. snow per year", "city"])
# print(empDfObj.dtypes)

In [17]:
# Comparator test
# d = {'col1': [1, 2, 3,4,5,21], 'col2': [3, 4,7,8,9,1]}
# df = pd.DataFrame(data=d)
# m = df["col1"].mean()
# xy = df.reset_index()[df.columns].values.tolist()
# df

In [18]:
# print(xy)
# print(m)

In [19]:
# sorted(xy, key=lambda x: abs(x[0] - m), reverse = True)
# xy

In [20]:
# t = map({xy.split(0), xy.split(0)},xy)
# t

In [21]:
# Sort by key
# import statistics

# import operator
# x = {1: 21, 3: 1, 4: 2, 2: 3, 0: 4, 11: 5}
# print(x.items())
# m1 = statistics.mean(x.values())
# print(m1)
# sorted_x = sorted(x.items(), key=lambda x: abs(x[1] - m1), reverse=True)
# print(sorted_x)

In [22]:
# a = [10.0, 9.8, 8.0, 7.8, 7.7, 7.0, 6.0, 5.0, 4.0, 2.0, 2.0]
# rank = lambda arr: list(map(lambda i: sorted(arr).index(i)+1, arr))
# rank(a)

In [23]:
# x = ['bottom25%', 'middle50%', 'top25%']
# res = "".join([i for i in x if "middle" in i])
# res = any("middle" in string for string in x)
# op =""
# flag=True

# for i in range(0,3):
#     if (res and flag):
#         flag = False
#         op = x[i]
#         print("Middle executed")
#     if x[i].startswith("top"):
#         print("Tiop executed")
#     elif x[i].startswith("bottom"):
#         print("Bottom executed executed")
# print(res)

In [24]:
# 129 >= 173 - 43.25


In [25]:
# def abc():
#     return "a", [1,2,3]

# x, y = abc()
# print(x)
# print(y)

In [26]:
# "m name is {city}'s".replace("{city}'s", "New York's")

In [27]:
# import re
# s = "The average amount spent per student is relatively high at {expend, per student $}. This places {city} {rank} on our city list in terms of eductaional priorities"
# x = re.findall("[{][\w.,$&-/(/)/\s]{0,}[}]", s)
# print(x)
# for i in x:
#     print(i[1:-1])
#     s = s.replace(i, str(cityData[np.logical_and(cityData["city"] == "Abilene", cityData["state_id"] == "TX")][i[1:-1]].values[0]))
# #     print(i[1:-1])
# #     p = s.find(i)
# #     print(p)
# #     s = s.replace(i, str(100))
# # print(s)

In [28]:
# x = cityData[cityData["city"] == "New York"]["population"]
# print(int(x))

In [29]:
# cityData[cityData["city"] == "New York"]["population"]

In [30]:
# (cityData[np.logical_and(cityData["city"] == "Albany", cityData["state_id"] == "GA")]["2019 job opportunity"].values)[0]

In [31]:
# for cityIndex in range(len(cityData)):
#     city = (cityData.iloc[[cityIndex]]).values
#     print(city[0][1])
#     break

In [32]:
# print(datetime.now())

In [33]:
# citylistData = (cityData.iloc[[1]]).values
# citylistData

In [34]:
# cityData[["city","population_rank"]]

In [35]:
# cityData[np.logical_and(cityData["city"] == "Albuquerque", cityData["state_id"] == "NM")]["avg. rainfall per year (mm)_rank"].values[0]

In [36]:
# def rescue_code(function):
#     import inspect
#     get_ipython().set_next_input("".join(inspect.getsourcelines(function)[0]))

In [37]:
# rescue_code(fetchAirQualityKey)

In [38]:
# s = "agflF AWLIDFHAWDN FCLAIWFHAM WFCLAWBC \n wgdbf liaudshfbmaw fkub \n aldjsfhlawf. \n"
# print(s.replace("\n", ""))

In [39]:
# Yuma has a low volume of job opportunities, with only about 61,940 positions available. 
# In Yuma, goods and services have average prices compared to other cities. The cost index is 82.3, putting Yuma 59th on our city list.
# Housing in Yuma is reasonably priced. The index is 83.9, putting it 62nd on our list of cities. 
# The weather in Yuma is very dry, with an annual rainfall of only 2.8 inches. This city is 174th in rainfall on our list of cities. 

# There are currently about 61,940 employment opportunites here, making Yuma the 140th on our city list to find a job.  Goods and services in Yuma are generally have average prices, with an index of 82.3, which ranks the city as having the 57th highest and the 59th lowest prices of the cities on our list.  Housing in Yuma is about average, with the housing index of 83.9 ranking the city 62nd on our city list.  Yuma's average commute time is pretty long: about 15.8 minutes. This is the 19th longest commute time on our city list.  The weather in Yuma is very dry, with an annual rainfall of only 2.8 inches. This city is 174th in rainfall on our list of cities.

In [40]:
# def test_meansort():
#     x = main.meansort(main.cityData["winter low"], df[["Name", "winter low"]])
#     print(x)
# test_meansort()

In [41]:
# cityData["avg. rainfall per year (inch)"].mean()

In [42]:
# if "$" in "{sdifau asdif $}":
#     print(True)

In [43]:
# cityData.columns

In [44]:
# cityData["winter low_rank"].nunique()